# Exercise 10

Diese Woche werden wir uns mit Matrixfaktorisierung beschäftigen.
Dazu verwenden wir einen Datensatz, der Nachrichten-Artikel enthält. Basierend auf den Titeln und Beschreibungen der Nachrichten sollen sie mögliche Themen/Topics ermitteln, unter denen die Artikel zusammengefasst bzw. gruppiert werden können.

Die Daten stammen von [kaggle](https://www.kaggle.com/datasets/rmisra/news-category-dataset/discussion/438853).

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import numpy as np
from numpy.typing import ArrayLike
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
def load_data(data_path: str) -> list:
    """Loads a dataset of news articles from a file containing multiple json objects.
    Each object is expected to at least contain the keys 'headline' and 'short_description'.
    
    Returns a list of joined headlines and short descriptions for each article (object).

    Args:
        data_path (str): The path to the json file containing the news articles.

    Returns:
        list: A list of strings where each string is a concatenation of the headline and short description of a news article.
    """
    with open(data_path, 'r') as f:
        json_data = [json.loads(line) for line in f]
    data = [entry['headline'] + ' ' + entry["short_description"] for entry in json_data]
    return data

In [3]:
def load_stop_words(file_path: str) -> set:
    """Load stop words from a file where each stop word is separated by a newline character.

    Args:
        file_path (str): The path to the file containing the stop words.

    Returns:
        set: A set of the stop words stripped of any leading/trailing whitespace.
    """
    with open(file_path, 'r') as f:
        return {line.strip() for line in f}

In [ ]:
data = load_data("./News_Category_Dataset_v3.json")
stop_words = load_stop_words("./stopwords-en.txt")

print("First 10 sentences:")
for line in data[:10]: print(line)

## Aufgabe 1

Zuerst wollen wir die Daten bereinigen, damit die Vorhersage der Themen besser funktioniert.
Dazu konvertieren wir die Wörter in jedem Artikel zu Kleinbuchstaben, entfernen Satzzeichen und Zahlen, und entfernen *stopwords*.

Die *stopwords* für diese Aufgabe wurden aus dieser [Quelle](https://github.com/stopwords-iso/stopwords-en/blob/master/stopwords-en.txt) entnommen.

Implementieren Sie die Funktion `clean_sentence`, die eine Liste von Datenpunkten und eine Menge von *stopwords* übergeben bekommt und die Datenpunkte bereinigt zurückgibt.

Die Funktion wird auf Moodle getestet.

In [5]:
import re

def clean_sentence(sentence: str, stop_words: set) -> str:
    """Cleans a sentence (string), by converting it to lowercase and removing numbers,
    punctuation, and stop words.

    Args:
        sentence (str): The original sentence.
        stop_words (set): A set of stop words.

    Returns:
        str: The cleaned sentence.
    """
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove numbers and punctuation (keep only letters and spaces)
    sentence = re.sub(r'[^\w\s]', ' ', sentence)  # Replace punctuation with a space
    sentence = re.sub(r'\d+', '', sentence)  # Remove numbers
    # Remove extra spaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    # Remove stop words
    words = sentence.split()
    cleaned_words = [word for word in words if word not in stop_words]
    # Join cleaned words into a sentence
    return ' '.join(cleaned_words)

In [ ]:
data = [clean_sentence(sentence, stop_words) for sentence in data]

print("First 10 cleaned sentences:")
for line in data[:10]: print(line)

## Featurization (keine Aufgabe)

Wir wollen nun die TF-IDF-Matrix bestimmen, die wir später faktorisieren, um die Topics zu modelieren.

Diese Art der Feature-Berechnung kennen Sie bereits aus einer vorherigen Übung und wird daher vorgegeben.

In [7]:
def compute_tfidf(data: list) -> tuple[ArrayLike, ArrayLike]:
    """Computes the TF-IDF matrix of a text corpus.
    Returns the TF-IDF scores and the associated words.

    Args:
        data (list): A list of strings where each string represents a sentence where words
        are seperated by whitepsace.

    Returns:
        ArrayLike: The TF-IDF matrix of the data.
        ArrayLike: The feature names associated with the TF-IDF matrix.
    """
    vectorizer = TfidfVectorizer()

    tfidf = vectorizer.fit_transform(data)
    words = vectorizer.get_feature_names_out()

    return tfidf, words

In [8]:
tfidf, words = compute_tfidf(data)

## Aufgabe 2

Nun soll die eigentliche Matrixfaktorisierung durchgeführt werden, um eine Topic-Zuordnung für die verschiedenen Artikel zu bestimmen.

Implementieren Sie die Funktion `compute_matrix_factorization`, diese soll die TF-IDF-Matrix für alle Artikel und die Anzahl der Topics übergeben bekommen und die Matrizen $U$ und $V^\top$ berechnen und zurückgeben.

Diese Funktion wird auf Moodle abgefragt.

In [9]:
from sklearn.decomposition import NMF
from numpy.typing import ArrayLike
import numpy as np

def compute_matrix_factorization(tfidf: ArrayLike, n_topics: int = 2) -> tuple[ArrayLike, ArrayLike]:
    """Computes the matrix factorization (U @ V.T) of the TF-IDF matrix using NMF.

    Args:
        tfidf (ArrayLike): The TF-IDF matrix.
        n_topics (int, optional): The number of topics to consider during matrix factorization. Defaults to 2.

    Returns:
        tuple[ArrayLike, ArrayLike]: The U and V.T matrices of the factorization.
    """
    # Ensure the TF-IDF matrix is non-negative and in dense format
    if isinstance(tfidf, np.ndarray):
        dense_tfidf = tfidf
    else:
        dense_tfidf = tfidf.toarray()  # Convert sparse matrix to dense format

    if np.any(dense_tfidf < 0):
        raise ValueError("TF-IDF matrix contains negative values. Ensure the input is non-negative.")

    # Initialize and perform Non-Negative Matrix Factorization (NMF)
    nmf = NMF(
        n_components=n_topics,
        init='nndsvda',  # Suitable initialization for sparse/non-negative matrices
        random_state=42,
        max_iter=300  # Default, but explicitly stated for clarity
    )
    U = nmf.fit_transform(dense_tfidf)  # U matrix (document-topic representation)
    V_T = nmf.components_  # V^T matrix (topic-term representation)

    return U, V_T


In [ ]:
n_topics = 3
U, V_T = compute_matrix_factorization(tfidf, n_topics)

## Aufgabe 3

Nachdem die Matrixfaktorisierung nun erfolgreich berechnet wurde, kann das Ergebnis dieser Berechnungen genutzt werden, um die Topics der Nachrichten zu bestimmen.

Implementieren Sie hierzu die Funktion `get_sentence_categories`, die die Matrizen $U$ und $V^\top$ übergeben bekommt und für jeden Artikel das zugehörige Topic bestimmt.

**Hinweis**: Die Funktion benötigt nur eine der beiden Matrizen, um die gesuchte Information zu bestimmen. Denken Sie selber darüber nach, wo diese Information zu finden ist.

Diese Funktion wird auf Moodle abgefragt.

In [11]:
import numpy as np
from numpy.typing import ArrayLike

def get_sentence_categories(U: ArrayLike, V_T: ArrayLike) -> ArrayLike:
    """Finds the category of each sentence based on the U matrix.

    Args:
        U (ArrayLike): The U matrix of the matrix factorization.
        V_T (ArrayLike): The V_T matrix of the matrix factorization.

    Returns:
        ArrayLike: The index of the topic each sentence belongs to.
    """
    # Determine the topic with the highest weight for each article (row in U)
    sentence_categories = np.argmax(U, axis=1)
    return sentence_categories


In [ ]:
sentence_categories = get_sentence_categories(U, V_T)
print("Categories assigned to some sentences:")
for i, sentence in enumerate(data[:10]):
    print(f"Sentence: '{sentence}' - Category: {sentence_categories[i]}")

## Aufgabe 4

Zusätzlich können wir die Topics anhand der Wörter charakterisieren, die mit einem Topic stark assoziiert sind.

Implementieren Sie hierzu die Funktion `get_most_important_topic_words`, die die Matrizen $U$ und $V^\top$, die Namen der Features `words` und die Anzahl der Wörter pro Topic `n_words` übergeben bekommt und für jedes Topic die `n_words` wichtigsten Wörter bestimmt.

**Hinweis**: Die Funktion benötigt nur eine der beiden Matrizen, um die gesuchte Information zu bestimmen. Denken Sie selber darüber nach, wo diese Information zu finden ist.

Diese Funktion wird auf Moodle abgefragt.

In [13]:
import numpy as np
from numpy.typing import ArrayLike

def get_most_important_topic_words(U: ArrayLike, V_T: ArrayLike, words: ArrayLike, n_words: int) -> list[list[str]]:
    """Return the most important words for each topic.

    Args:
        U (ArrayLike): The U matrix (m x k) of the non-negative matrix factorization.
        V_T (ArrayLike): The V.T matrix (k x n) of the non-negative matrix factorization.
        words (ArrayLike): The feature names (n x 1) associated with the TF-IDF matrix.
        n_words (int): The number of words to return for each topic.

    Returns:
        list[list[str]]: A list of lists of strings where each inner list contains
        the most important words for a topic sorted from most important to least important.
    """
    topic_words = []

    # Für jedes Topic
    for i in range(V_T.shape[0]):
        # Extrahiere die Gewichtungen für das aktuelle Topic
        topic_weights = V_T[i]
        
        # Holen Sie sich die Indizes der n wichtigsten Wörter
        top_indices = np.argsort(topic_weights)[::-1][:n_words]
        
        # Ordne die Wörter den Indizes zu und füge sie zur Liste hinzu
        topic_words.append([words[idx] for idx in top_indices])
    
    return topic_words


In [ ]:
print("Most important words for each topic:")
for topic_idx, topic in enumerate(get_most_important_topic_words(U, V_T, words, 20)):
    print(f"Topic {topic_idx}: {topic}")

## Word Cloud (keine Aufgabe)

Alternativ können Sie die wichtigsten Wörter pro Topic auch als *word cloud* darstellen.

In [15]:
def plot_word_cloud(U: ArrayLike, V_T: ArrayLike, topic_idx: int, words: ArrayLike):
    """Plots a word cloud of the most important words for a given topic.

    Args:
        U (ArrayLike): The U matrix (m x k) of the non-negative matrix factorization.
        V_T (ArrayLike): The V.T matrix (k x n) of the non-negative matrix factorization.
        topic_idx (int): The index of the topic to plot the word cloud for.
        words (ArrayLike): The feature names (n x 1) associated with the TF-IDF matrix.
    """
    topic = V_T[topic_idx]
    plt.figure(figsize=(10, 5))
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(zip(words, topic)))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Topic {topic_idx}')
    plt.show()

In [ ]:
for topic_idx in range(min(n_topics, 10)):
    plot_word_cloud(U, V_T, topic_idx, words)

## Aufgabe 5

Zuletzt wollen wir betrachten, welche Wörter häufig in Artikeln eines bestimmten Topics gefunden werden.

Implementieren Sie hierzu die Funktion `visualize_words_by_topic`, die die Matrizen $U$ und $V^\top$, die Namen der Features `words`, die Indizes des latenten Raums, die auf der $x$ und $y$ Achse visualisiert werden sollen `topic_x`, `topic_y` und eine Anzahl an Wörtern `n_words` übergeben bekommt und die Wörter mit dem größten Einfluss auf die Topics visualisiert.

In [17]:
def visualize_words_by_topic(U: ArrayLike, V_T: ArrayLike, words: ArrayLike, topic_x:int=0, topic_y:int=1, n_words:int=50):
    """Visualizes the contributions of words to two topics in a scatter plot.

    Args:
        U (ArrayLike): The U matrix (m x k) of the non-negative matrix factorization
        V_T (ArrayLike): The V.T matrix (k x n) of the non-negative matrix factorization.
        words (ArrayLike): The feature names (n x 1) associated with the TF-IDF matrix.
        topic_x (int, optional): The latent space index visualized on the x-axis. Defaults to 0.
        topic_y (int, optional): The latent space index visualized on the y-axis. Defaults to 1.
        n_words (int, optional): The number of words that should be visualized. Defaults to 50.
    """
    # Get contributions for the chosen topics only
    # Combine contributions from both topics into a tuple (x, y) for sorting
    contributions = np.array([(V_T[topic_x, i], V_T[topic_y, i]) for i in range(V_T.shape[1])])
    
    # Get the top N indices based on the contribution of the specified topics
    top_indices = np.argsort(np.linalg.norm(contributions, axis=1))[-n_words:]
    
    # Get the corresponding words and their contributions for the selected topics
    top_words = words[top_indices]
    
    # Get the contributions for the chosen topics
    x_values = V_T[topic_x, top_indices]  # Contributions to topic X
    y_values = V_T[topic_y, top_indices]  # Contributions to topic Y

    # Create the scatter plot
    plt.figure(figsize=(12, 12))
    plt.scatter(x_values, y_values, alpha=0.7)

    # Annotate each point with the corresponding word
    for i, word in enumerate(top_words):
        plt.annotate(word, (x_values[i], y_values[i]), textcoords="offset points", xytext=(0, 5), ha='center')

    plt.title(f"Word Contributions to Topic {topic_x} vs Topic {topic_y}")
    plt.xlabel(f"Contribution to Topic {topic_x}")
    plt.ylabel(f"Contribution to Topic {topic_y}")
    plt.grid()
    plt.show()

In [ ]:
visualize_words_by_topic(U, V_T, words, topic_x=0, topic_y=1, n_words=50)